In [1]:
!pip install pyunpack
!pip install patool
# Extracting the .7z file, as the given file is in .7z format and the notebook is running on kaggle
import os
from pyunpack import Archive
os.system('apt-get install p7zip')
import shutil
if not os.path.exists('/kaggle/working/train/'):
    '''If already, some file is created don't make it
    '''
    os.makedirs('/kaggle/working/train/')

# Extracting the .7z file 
Archive('../input/tensorflow-speech-recognition-challenge/train.7z').extractall('/kaggle/working/train/')


# Checking the number of each file
import os
path = os.listdir('./train/train/audio/')
size = {}
for i in path:
      size[i] = len(os.listdir('./train/train/audio/'+i))
print(size)

     |████████████████████████████████| 77 kB 940 kB/s 
{'house': 1750, 'six': 2369, 'wow': 1745, 'eight': 2352, 'cat': 1733, 'one': 2370, 'four': 2372, 'seven': 2377, 'bed': 1713, 'happy': 1742, '_background_noise_': 7, 'zero': 2376, 'no': 2375, 'two': 2373, 'three': 2356, 'yes': 2377, 'off': 2357, 'on': 2367, 'bird': 1731, 'tree': 1733, 'left': 2353, 'five': 2357, 'down': 2359, 'sheila': 1734, 'up': 2375, 'right': 2367, 'marvin': 1746, 'go': 2372, 'stop': 2380, 'dog': 1746, 'nine': 2364}


In [2]:
import librosa # For loading the audio file

labels_to_consider = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
unknown = [i for i in os.listdir('./train/train/audio') if i not in labels_to_consider and i != '_background_noise_' ]

def label_encoder(directory):
    '''
    Input: labels -> list of labels to be considered
           directory -> the folder in which the examples for all the dataset is present
           
    Output : list containing the entry as: (sample,gruop number)
    '''
    i = 0
    label_encoder ={}
    labelled_wave = []
    training_data = []
    for label in labels_to_consider:
        path = os.path.join(directory,label)
        label_encoder[label] = i    # Label to encoder
        print("The current label is: "+str(label)+" of: "+str(i))
        i+=1
        for audio_file in os.listdir(path):
            if audio_file.endswith('.wav'):
                samples, sample_rate = librosa.load(os.path.join(os.path.join(directory,label),audio_file))
                samples = librosa.resample(samples,sample_rate,8000)
            if len(samples)!=8000:
                continue
            else:
                labelled_wave.append([samples,label])
                training_data.append(samples)
    for label in unknown:
                print("The current label is: "+str("unknown"+" of: "+str(i)))
                label_encoder['unknown'] = i
                path = os.path.join(directory,label)
                for audio_file in os.listdir(path):
                    if audio_file.endswith('.wav'):
                        samples, sample_rate = librosa.load(os.path.join(os.path.join(directory,label),audio_file))
                        samples = librosa.resample(samples,sample_rate,8000)
                    if len(samples)!=8000:
                        continue
                    else:
                        labelled_wave.append([samples,'unknown'])
                        training_data.append(samples)
    return labelled_wave,training_data,label_encoder
labelled_wave,training_data,label_encoder = label_encoder('./train/train/audio')

The current label is: yes of: 0
The current label is: no of: 1
The current label is: up of: 2
The current label is: down of: 3
The current label is: left of: 4
The current label is: right of: 5
The current label is: on of: 6
The current label is: off of: 7
The current label is: stop of: 8
The current label is: go of: 9
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current label is: unknown of: 10
The current l

In [3]:
import numpy as np
import random
def get_random_noise():
    ''' This function will be useful for getting the random noise
    '''
    audios = os.listdir('./train/train/audio/_background_noise_')
    noise = random.randint(0,len(audios)-1)
    noise,sr = librosa.load('./train/train/audio/_background_noise_/'+audios[noise])
    noise = librosa.resample(noise,sr,8000)
    start =random.randint(0,noise.shape[0]-8000-1)
    return noise[start:start+8000]


def mix_audio(data,ratio = 0.1):
    ''' This function will mix the original audio with the background noise
    '''
    noise = get_random_noise()
    final_data = []
    for i,j in enumerate(data):
        final_data.append(j + (ratio*noise))
    return final_data
final_data = mix_audio(training_data)

In [4]:
labels = [i[1] for i in labelled_wave]
def remove_not_equal_length(data,labels):
    
    '''If some array has some inadequate length, we would remove it
    '''
    final_data = []
    for i,j in zip(data,labels):
        if len(i)!=8000:
            continue
        else:
            final_data.append([i,j])
    return final_data
dataset = remove_not_equal_length(final_data,labels)

In [5]:
# Traininng data = wav_file
# Labels = target
wave_file = [i[0] for i in dataset]
wave_file = np.reshape(np.array(wave_file),(-1,8000,1))
target = [label_encoder[i[1]] for i in dataset]
target = np.reshape(target,(-1,1))
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(wave_file,target,test_size= 0.15,random_state=98,shuffle=True)
import keras
y_train = keras.utils.to_categorical(y_train, len(set(labels))+1)
y_test = keras.utils.to_categorical(y_test, len(set(labels))+1)
training_labels = [np.argmax(i) for i in y_train]
testing_labels = [np.argmax(i) for i in y_test]
train  = X_train.reshape(-1,8000)
test  = X_test.reshape(-1,8000)

## KNN

In [6]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(train,training_labels)

KNeighborsClassifier()

In [7]:
from sklearn.metrics import classification_report,accuracy_score,f1_score
print(classification_report(testing_labels,model.predict(test)))
print("The accuracy score is:",accuracy_score(testing_labels,model.predict(test)))
#print("The F! score is:",f1_score(testing_labels,classifier.predict(test)))

              precision    recall  f1-score   support

           0       0.08      0.07      0.08       338
           1       0.08      0.05      0.06       307
           2       0.07      0.06      0.06       300
           3       0.06      0.01      0.02       336
           4       0.03      0.02      0.02       319
           5       0.09      0.03      0.04       306
           6       0.11      0.02      0.04       330
           7       0.17      0.06      0.09       310
           8       0.10      0.03      0.04       324
           9       0.09      0.03      0.05       323
          10       0.65      0.85      0.74      5545

    accuracy                           0.56      8738
   macro avg       0.14      0.11      0.11      8738
weighted avg       0.44      0.56      0.49      8738

The accuracy score is: 0.5551613641565576


In [8]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model,open(filename, 'wb'))